In [3]:
!ls -la ~/Data/Instacart

total 732888
drwxr-xr-x 3 ike ike      4096 Feb 15 23:16 .
drwxr-xr-x 3 ike ike      4096 Feb 10 13:45 ..
-rw-rw-rw- 1 ike ike      2603 Jun 10  2017 aisles.csv
-rw-rw-rw- 1 ike ike       270 Jun 10  2017 departments.csv
-rw-r--r-- 1 ike ike  27681062 Feb 15 23:17 log.prior.small.csv
-rw-r--r-- 1 ike ike   1191183 Feb 15 23:17 log.train.small.csv
drwxr-xr-x 3 ike ike      4096 Feb 10 21:57 mk
-rw-rw-rw- 1 ike ike 577550706 Jun 10  2017 order_products__prior.csv
-rw-rw-rw- 1 ike ike  24680147 Jun 10  2017 order_products__train.csv
-rw-rw-rw- 1 ike ike 108968645 Jun 10  2017 orders.csv
-rw-r--r-- 1 ike ike   5238924 Feb 15 23:16 orders.small.csv
-rw-rw-rw- 1 ike ike   2166953 Jun 10  2017 products.csv
-rw-rw-rw- 1 ike ike   1475693 Jun 10  2017 sample_submission.csv
-rw-r--r-- 1 ike ike   1475693 Feb 15 23:17 sample_submission.small.csv


In [4]:
!rm ~/Data/Instacart/mk -rf

In [1]:
import pandas as pd
import numpy as np
from setting import *

orders = pd.read_csv(DATA + '/orders.small.csv', dtype={
        'order_id': np.int32,
        'user_id': np.int32,
        'eval_set': 'category',
        'order_number': np.int16,
        'order_dow': np.int8,
        'order_hour_of_day': np.int8,
        'days_since_prior_order': np.float32}, engine='c').\
        rename(columns = {'order_dow': 'order_week', 'order_hour_of_day': 'order_hour', 'days_since_prior_order': 'order_gap'})

In [3]:
log=pd.concat([
    pd.read_csv(DATA + '/log.train.small.csv', dtype={
            'order_id': np.int32,
            'product_id': np.uint16,
            'add_to_cart_order': np.int16,
            'reordered': np.int8}, usecols=['order_id', 'product_id', 'reordered'], engine='c'),
    pd.read_csv(DATA + '/log.prior.small.csv', dtype={
            'order_id': np.int32,
            'product_id': np.uint16,
            'add_to_cart_order': np.int16,
            'reordered': np.int8}, usecols=['order_id', 'product_id', 'reordered'], engine='c')],
    ignore_index=True)
log.sort_values(by=['order_id', 'product_id'], inplace=True)
log = log.reset_index(drop=True)

In [2]:
orders.head()

,order_id,user_id,eval_set,order_number,order_week,order_hour,order_gap
0,1374495,3,prior,1,1,14,NaN
1,444309,3,prior,2,3,19,9.0
2,3002854,3,prior,3,3,16,21.0
3,2037211,3,prior,4,2,18,20.0
4,2710558,3,prior,5,0,17,12.0


In [3]:
orders['order_id_t-1'] = orders.groupby('user_id').order_id.shift(1)
orders['order_id_t-2'] = orders.groupby('user_id').order_id.shift(2)
orders['order_id_t-3'] = orders.groupby('user_id').order_id.shift(3)

In [4]:
order_hour = orders[['order_id', 'order_hour']]

In [6]:
order_prevhour = orders.merge(order_hour.add_suffix('_t-1'), on='order_id_t-1', how='left').\
merge(order_hour.add_suffix('_t-2'), on='order_id_t-2', how='left').\
merge(order_hour.add_suffix('_t-3'), on='order_id_t-3', how='left')

In [9]:
order_prevhour['order_delta_hour_t-1'] = order_prevhour.order_hour-order_prevhour['order_hour_t-1']
order_prevhour['order_delta_hour_t-2'] = order_prevhour.order_hour-order_prevhour['order_hour_t-2']
order_prevhour['order_delta_hour_t-3'] = order_prevhour.order_hour-order_prevhour['order_hour_t-3']

In [10]:
order_prevhour.head()

,order_id,user_id,eval_set,order_number,order_week,order_hour,order_gap,order_id_t-1,order_id_t-2,order_id_t-3,order_hour_t-1,order_hour_t-2,order_hour_t-3,order_delta_hour_t-1,order_delta_hour_t-2,order_delta_hour_t-3
0,1374495,3,prior,1,1,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,444309,3,prior,2,3,19,9.0,1.3745e+06,NaN,NaN,14.0,NaN,NaN,5.0,NaN,NaN
2,3002854,3,prior,3,3,16,21.0,444309,1.3745e+06,NaN,19.0,14.0,NaN,-3.0,2.0,NaN
3,2037211,3,prior,4,2,18,20.0,3.00285e+06,444309,1.3745e+06,16.0,19.0,14.0,2.0,-1.0,4.0
4,2710558,3,prior,5,0,17,12.0,2.03721e+06,3.00285e+06,444309,18.0,16.0,19.0,-1.0,1.0,-2.0


In [12]:
order_prevhour = order_prevhour.drop(columns=['user_id', 'eval_set', 'order_number', 'order_week', 'order_hour', 
                             'order_gap', 'order_id_t-1', 'order_id_t-2', 'order_id_t-3',
                             'order_hour_t-1', 'order_hour_t-2', 'order_hour_t-3']).\
               set_index('order_id')

In [13]:
order_prevhour.to_csv('order_prev_hour.csv')